# Importing stuff

In [ ]:
import great_expectations as gx

In [ ]:
context = gx.get_context()
context = context.convert_to_file_context()


# Create DataSource and Assets

In [ ]:
%env PG_HOST=localhost
%env PG_PORT=5432
%env PG_USER=postgres
%env PG_PASSWORD=postgres
%env PG_DATABASE=postgres

In [ ]:
pg_datasource = context.sources.add_or_update_sql(name="postgres", connection_string="postgresql://${PG_USER}:${PG_PASSWORD}@${PG_HOST}:${PG_PORT}")


In [ ]:
table_asset = pg_datasource.add_table_asset(name="movies", table_name="movies_metadata", schema_name="public")
table_asset.add_splitter_year_and_month(column_name="release_date")

# Batch Requests

In [ ]:

batch_request = table_asset.build_batch_request(options={"year": 2010})
table_asset.get_batch_list_from_batch_request(batch_request)


# Expectations

In [ ]:


context.add_or_update_expectation_suite(expectation_suite_name="movies" )

In [ ]:
validator = context.get_validator(batch_request=batch_request, expectation_suite_name="movies")
validator.head()

## Expectations Gallery
https://greatexpectations.io/expectations/?filterType=undefined&gotoPage=undefined&showFilters=undefined&viewType=undefined

In [ ]:
validator.expect_column_distinct_values_to_equal_set(column="status", value_set=("Rumored", "Released"))

In [ ]:
validator.save_expectation_suite()

# Onboarding Data Assistant

In [ ]:
exclude_column_names = []

data_assistant_result = context.assistants.onboarding.run(
    batch_request=batch_request,
    exclude_column_names=exclude_column_names,
)

expectation_suite = data_assistant_result.get_expectation_suite(
    expectation_suite_name="movies"
)
context.add_or_update_expectation_suite(expectation_suite=expectation_suite)

# Checkpoints

In [ ]:
batch_request_year_2011 = table_asset.build_batch_request({'year' : 2011})

In [ ]:
checkpoint = context.add_or_update_checkpoint(
    name=f"movies_checkpoint",
    batch_request=batch_request_year_2011,
    validations=[
        {
            "batch_request": None,
            "expectation_suite_name": "movies",
        }
    ],
)
checkpoint_result = checkpoint.run()



In [ ]:
batch_request_year_2012 = table_asset.build_batch_request({'year' : 2012})
checkpoint = context.get_checkpoint("movies_checkpoint")
checkpoint.run(batch_request=batch_request_year_2012)